In [1]:
# %%
%load_ext autoreload
%autoreload 2

# %%
import os
import yaml
import json

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt

In [2]:

ci = 0.95
alpha = (1 - ci) / 2


def half_ci(group):
    data = group.dropna().to_numpy()
    sem = stats.sem(data)
    t2 = stats.t.ppf(1 - alpha, len(data) - 1) - stats.t.ppf(alpha, len(data) - 1)
    return sem * (t2 / 2)


def lower_ci(group):
    data = group.dropna().to_numpy()
    sem = stats.sem(data)
    mean = data.mean()
    t = stats.t.ppf(alpha, len(data) - 1)
    return mean + sem * t


def upper_ci(group):
    data = group.dropna().to_numpy()
    sem = stats.sem(data)
    mean = data.mean()
    t = stats.t.ppf(1 - alpha, len(data) - 1)
    return mean + sem * t


def walklevel(some_dir, level=1):
    some_dir = some_dir.rstrip(os.path.sep)
    assert os.path.isdir(some_dir)
    num_sep = some_dir.count(os.path.sep)
    for root, dirs, files in os.walk(some_dir):
        yield root, dirs, files
        num_sep_this = root.count(os.path.sep)
        if num_sep + level <= num_sep_this:
            del dirs[:]

from collections.abc import MutableMapping

def flatten(dictionary, parent_key='', separator='_'):
    items = []
    for key, value in dictionary.items():
        new_key = parent_key + separator + key if parent_key else key
        if isinstance(value, MutableMapping):
            items.extend(flatten(value, new_key, separator=separator).items())
        else:
            items.append((new_key, value))
    return dict(items)


def query(data_frame, query_string):
    if query_string == "all":
        return data_frame
    return data_frame.query(query_string)

In [3]:
root_path = "logs_eval"

configs = {}
results = {}
rows = []

# traverse root directory, and list directories as dirs and files as files
for root, dirs, files in walklevel(root_path, level=2):
    path = root.split(os.sep)
    try:
        if len(path) < 3:
            continue

        with open(os.path.join(root, "config.yaml"), 'r') as stream:
            config = yaml.safe_load(stream)
        with open(os.path.join(root,'results.json')) as json_data:
            result = json.load(json_data)
            json_data.close()
    except:
        continue
    # print('config', config)
    # print('results', result)
    # rows.append({**config, **result})
    rows.append({**flatten(config), **flatten(result)})

runs_df = pd.DataFrame(rows)

In [4]:
runs_df

,conditioning_delta,conditioning_n_corrector,conditioning_name,conditioning_p_cond,dataset_image_size,dataset_name,dataset_num_channels,diffusion_num_steps,likelihood_name,likelihood_pad_value,...,lpips_mean,mse_median,lpips_median,mse_std,lpips_std,fid,conditioning_noise,conditioning_start_fraction,conditioning_gamma,conditioning_update_rule
0,0.1,0,amortized,0.9,64,flowers,3,1000,outpainting,-2,...,0.253944,0.317296,0.245443,0.157875,0.060657,None,NaN,NaN,NaN,NaN
1,0.1,0,amortized,0.9,64,flowers,3,1000,outpainting,-2,...,0.251263,0.275953,0.250575,0.158769,0.062681,None,NaN,NaN,NaN,NaN
2,0.1,0,amortized,0.9,64,flowers,3,1000,outpainting,-2,...,0.248992,0.280449,0.248412,0.162965,0.064261,None,NaN,NaN,NaN,NaN
3,0.1,0,amortized,0.9,64,flowers,3,1000,outpainting,-2,...,0.249482,0.311888,0.233649,0.167900,0.059234,None,NaN,NaN,NaN,NaN
4,0.1,0,amortized,0.9,64,flowers,3,1000,outpainting,-2,...,0.259861,0.308657,0.255696,0.187240,0.063850,None,NaN,NaN,NaN,NaN
5,0.1,0,replacement,NaN,64,flowers,3,1000,outpainting,-2,...,0.339170,0.231831,0.330761,0.142490,0.088437,None,True,0.5,NaN,NaN
6,0.1,0,replacement,NaN,64,flowers,3,1000,outpainting,-2,...,0.355367,0.249148,0.344532,0.137901,0.077156,None,True,0.5,NaN,NaN
7,0.1,0,replacement,NaN,64,flowers,3,1000,outpainting,-2,...,0.324017,0.252660,0.319062,0.154057,0.084674,None,True,0.5,NaN,NaN
8,0.1,0,replacement,NaN,64,flowers,3,1000,outpainting,-2,...,0.341028,0.230077,0.335218,0.144144,0.067953,None,True,0.5,NaN,NaN
9,0.1,0,replacement,NaN,64,flowers,3,1000,outpainting,-2,...,0.331489,0.253034,0.323274,0.130135,0.077730,None,True,0.5,NaN,NaN


In [5]:
# print(runs_df)
print(runs_df.columns)
# runs_df[['mse_mean', 'mse_std']]
params = ['conditioning_name', 'conditioning_noise', 'conditioning_start_fraction', 'conditioning_gamma']
params += ['conditioning_n_corrector', 'conditioning_delta']
metrics = ['mse_mean', 'lpips_mean']

criteria = [
    "`conditioning_delta` < 0.5"
]
criteria = ["all"] if criteria == [] else criteria
runs_df = query(runs_df, " & ".join(criteria))

# reducers = ["mean", "std", "sem", lower_ci, upper_ci, half_ci, "count"]
reducers = ["mean", 'std', half_ci, "count"]
metrics = metrics if isinstance(metrics, list) else [metrics]
metrics_to_agg = {key: reducers for key in metrics}
runs_df.groupby(params, dropna=False).agg(metrics_to_agg)

Index(['conditioning_delta', 'conditioning_gamma', 'conditioning_n_corrector',
       'conditioning_name', 'conditioning_start_fraction',
       'conditioning_update_rule', 'dataset_image_size', 'dataset_name',
       'dataset_num_channels', 'diffusion_num_steps', 'likelihood_name',
       'likelihood_pad_value', 'likelihood_patch_size', 'network_model_path',
       'network_num_channels', 'network_num_head_channels',
       'network_num_heads', 'network_num_res_blocks',
       'network_resblock_updown', 'network_use_scale_shift_norm',
       'testing_batch_size', 'testing_fid', 'testing_num_test', 'testing_seed',
       'training_batch_size', 'training_lr_end_warmup', 'training_lr_final',
       'training_lr_schedule', 'training_num_epochs', 'training_warmup_steps',
       'mse_mean', 'lpips_mean', 'mse_median', 'lpips_median', 'mse_std',
       'lpips_std', 'fid', 'conditioning_noise'],
      dtype='object')


/homes/ebm32/protein-diffusion/venv/lib/python3.9/site-packages/numpy/core/_methods.py:264: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/homes/ebm32/protein-diffusion/venv/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


mse_mean  \
                                                                                                                                           mean   
conditioning_name       conditioning_noise conditioning_start_fraction conditioning_gamma conditioning_n_corrector conditioning_delta             
reconstruction_guidance NaN                0.5                         0.1                0                        0.00                0.124471   
                                                                       0.5                0                        0.00                0.056701   
                                                                       1.0                0                        0.00                0.044486   
                                                                       2.0                0                        0.00                0.037070   
                                                                       5.0                0                        0.00                0.029354   
                                                                       10.0               0                        0.00                0.023598   
                                                                                          1                        0.01                0.025927   
                                                                                                                   0.05                0.023770   
                                                                                                                   0.10                0.024533   
                                                                                          2                        0.01                0.023848   
                                                                                                                   0.05                0.022601   
                                                                                                                   0.10                0.025041   
                                                                                          3                        0.01                0.023474   
                                                                                                                   0.05                0.023313   
                                                                                                                   0.10                0.030891   
                                                                                          5                        0.01                0.021634   
                                                                                                                   0.05                0.028905   
                                                                                                                   0.10                0.040424   
                                           0.7                         0.1                0                        0.00                0.074605   
                                                                       0.5                0                        0.00                0.046917   
                                                                       1.0                0                        0.00                0.039598   
                                                                       2.0                0                        0.00                0.033994   
                                                                       5.0                0                        0.00                0.029694   
                                                                       10.0               0                        0.00                0.027305   
                                           1.0                         0.1                0                        0.00                0.073492   
                   

In [68]:
runs_df[params + ['mse_mean', 'mse_std', 'lpips_mean', 'lpips_std']].sort_values(params)

,conditioning_name,conditioning_noise,conditioning_start_fraction,mse_mean,mse_std,lpips_mean,lpips_std
2,replacement,False,0.5,0.023227,0.016565,0.060793,0.036179
4,replacement,False,0.7,0.022955,0.017300,0.059910,0.035461
0,replacement,False,1.0,0.022992,0.017407,0.059903,0.035435
1,replacement,True,0.5,0.030158,0.031415,0.062168,0.050451
5,replacement,True,0.7,0.022648,0.021766,0.051020,0.028295
3,replacement,True,1.0,0.021295,0.019083,0.048991,0.027679
